In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import TextVectorization, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy


In [2]:
import pandas as pd

# Load dataset
df = pd.read_csv('training.csv')  # Replace with your dataset path

# Inspect the first few rows
print(df.head())


In [5]:
import pandas as pd

# Load dataset
df = pd.read_csv('training.csv')  # Replace with your dataset path

# Inspect the first few rows
print(df.head())


                                                text  label
0                            i didnt feel humiliated      0
1  i can go from feeling so hopeless to so damned...      0
2   im grabbing a minute to post i feel greedy wrong      3
3  i am ever feeling nostalgic about the fireplac...      2
4                               i am feeling grouchy      3


In [7]:
X = df['text']  # Sentences
y = df['label']  # Emotion labels


In [17]:
from sklearn.preprocessing import LabelBinarizer

# Define emotion labels
emotion_labels = ['Anger', 'Fear', 'Joy', 'Love', 'Sadness', 'Surprise']
label_binarizer = LabelBinarizer()
label_binarizer.fit(emotion_labels)

# Convert labels to one-hot encoding
y_one_hot = label_binarizer.transform([emotion_labels[i] for i in y])


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.3, random_state=42)


In [10]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_features = 10000  # Vocabulary size
sequence_length = 70  # Maximum sequence length

vectorizer = TextVectorization(
    max_tokens=max_features,
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True
)

# Fit the vectorizer on training data
vectorizer.adapt(X_train)


In [11]:
X_train_vectorized = vectorizer(X_train)
X_test_vectorized = vectorizer(X_test)


In [12]:
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy

num_labels = y_one_hot.shape[1]

input_layer = Input(shape=(sequence_length,), dtype=tf.int32, name='text_input')
embedding_layer = Embedding(input_dim=max_features, output_dim=128)(input_layer)
x = Conv1D(128, 5, activation='relu')(embedding_layer)
x = MaxPooling1D(pool_size=4)(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(num_labels, activation='sigmoid')(x)  # Sigmoid for multi-label classification

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[Accuracy()])


In [13]:
history = model.fit(
    X_train_vectorized,
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)


Epoch 1/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.0000e+00 - loss: 0.5048 - val_accuracy: 0.0000e+00 - val_loss: 0.4112
Epoch 2/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.0000e+00 - loss: 0.4056 - val_accuracy: 0.0000e+00 - val_loss: 0.4099
Epoch 3/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.0000e+00 - loss: 0.4054 - val_accuracy: 0.0000e+00 - val_loss: 0.4094
Epoch 4/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.0000e+00 - loss: 0.4010 - val_accuracy: 0.0000e+00 - val_loss: 0.4059
Epoch 5/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.0000e+00 - loss: 0.3953 - val_accuracy: 0.0000e+00 - val_loss: 0.3934
Epoch 6/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.0000e+00 - loss: 0.3680 - val_accuracy: 0.0000e+00 - val_loss: 0.3428
Epoch 7/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.0000e+00 - loss: 0.3009 - val_accuracy: 0.0000e+00 - val_loss: 0.2804
Epoch 8/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 7

In [14]:
loss, accuracy = model.evaluate(X_test_vectorized, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 4.4665e-05 - loss: 0.2041
Test Loss: 0.2034
Test Accuracy: 0.0000


In [18]:
# Function to predict emotion percentages
def predict_emotion(sentence):
    vectorized_text = vectorizer(tf.constant([sentence]))
    prediction = model.predict(vectorized_text)
    emotion_percentages = prediction[0] * 100  # Convert to percentage
    outstanding_emotion_index = np.argmax(prediction[0])
    outstanding_emotion = emotion_labels[outstanding_emotion_index]
    return emotion_percentages, outstanding_emotion

# Example prediction
sentence = "I feel really happy today!"
percentages, top_emotion = predict_emotion(sentence)
print(f"Emotion Percentages: {dict(zip(emotion_labels, percentages))}")
print(f"Outstanding Emotion: {top_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Emotion Percentages: {'Anger': 7.4262724, 'Fear': 52.642464, 'Joy': 6.430655, 'Love': 7.9768195, 'Sadness': 0.2590005, 'Surprise': 2.651053}
Outstanding Emotion: Fear
